In [1]:
# Specify CUDA device
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['JAX_CHECK_TRACER_LEAKS'] = 'True'

import jax
jax.config.update("jax_enable_x64", True)

# Check we're running on GPU
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

from jax import jit, grad 
import jax.numpy as jnp 
from jax.test_util import check_grads
import numpy as np 

import s2fft 

cpu


In [2]:
L = 16
sampling = "mw"
np.random.seed(1911851)
f_target = np.random.randn(2*L, 2*L-1)+1j*np.random.randn(2*L, 2*L-1)
flm_target = s2fft.forward_jax(f_target, L, sampling=sampling)
f_target = s2fft.inverse_jax(flm_target, L, sampling=sampling)
precomps = s2fft.generate_precomputes_jax(L, forward=True, sampling=sampling)

In [3]:
np.random.seed(130672510)
f = np.random.randn(2*L, 2*L-1) + 1j*np.random.randn(2*L, 2*L-1)

In [4]:
def func(f):
    flm = s2fft.forward_jax(f, L, reality=False, precomps=precomps,sampling=sampling)
    return jnp.sum(jnp.abs(flm-flm_target)**2)
grad_func = grad(func)

In [5]:
check_grads(func, (f,), order=1, modes=('rev'))